In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed, wait
import os

# File paths
input_csv = 'drug_refs_4.csv'

# Load full input CSV
#df = pd.read_csv(input_csv)

In [2]:
def get_text(doi):
    if pd.isna(doi):
        return ''
    try:
        response = requests.get(f'https://api.crossref.org/works/{doi}', timeout=10)
        if response.status_code == 200:
            data = response.json()
            text = data.get('message', {})
            return text
    except:
        return ''
    return ''

def process_row(idx, doi):
    return idx, get_references(doi)

In [68]:
# response = requests.get(f'https://api.crossref.org/works/{doi}', timeout=10)
df.iloc[0]['doi']

'10.1021/jm00143a012'

In [3]:
doi = "10.1016/j.bmcl.2019.03.021"
content = requests.get(f'https://api.crossref.org/works/{doi}', timeout=10).json()['message']
for k in content.keys():
    print(k + ": " + str(content[k]))

indexed: {'date-parts': [[2025, 10, 18]], 'date-time': '2025-10-18T08:10:31Z', 'timestamp': 1760775031280, 'version': 'build-2065373602'}
reference-count: 36
publisher: Elsevier BV
issue: 11
license: [{'start': {'date-parts': [[2019, 6, 1]], 'date-time': '2019-06-01T00:00:00Z', 'timestamp': 1559347200000}, 'content-version': 'tdm', 'delay-in-days': 0, 'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/'}, {'start': {'date-parts': [[2019, 6, 1]], 'date-time': '2019-06-01T00:00:00Z', 'timestamp': 1559347200000}, 'content-version': 'tdm', 'delay-in-days': 0, 'URL': 'https://www.elsevier.com/legal/tdmrep-license'}]
content-domain: {'domain': ['elsevier.com', 'sciencedirect.com'], 'crossmark-restriction': True}
short-container-title: ['Bioorganic &amp; Medicinal Chemistry Letters']
published-print: {'date-parts': [[2019, 6]]}
DOI: 10.1016/j.bmcl.2019.03.021
type: journal-article
created: {'date-parts': [[2019, 3, 19]], 'date-time': '2019-03-19T11:05:14Z', 'timestamp': 1552993514000}
page:

In [70]:
# doi = "10.1038/s41586-020-2649-2"
url = f"https://doi.org/{doi}"

headers = {
    "Accept": "text/plain"   # or "application/vnd.crossref.unixref+xml"
}

r = requests.get(url, headers=headers)


In [71]:
r.text

'Kawamoto, Y., Tomino, M., Hiramatsu, K., Oyama, Y., & Hayashi, Y. (2019). Benzothiophene derivatives as phosphodiesterase 10A (PDE10A) inhibitors: Hit-to-lead studies. Bioorganic &amp; Medicinal Chemistry Letters, 29(11), 1419â\x80\x931422. https://doi.org/10.1016/j.bmcl.2019.03.021\n'

In [4]:
import requests
from xml.etree import ElementTree as ET

def doi_to_pmid(doi):
    """Return PMID from a DOI using NCBI ESearch."""
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
		"db": "pubmed",
		"term": f"{doi}[doi]",
		"retmode": "xml"
	}
    r = requests.get(url, params=params)
    root = ET.fromstring(r.text)
    pmids = root.findall(".//Id")
    return pmids[0].text if pmids else None


def pmid_to_pmcid(pmid):
    """Return PMCID from a PMID using NCBI ELink."""
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"
    params = {
        "dbfrom": "pubmed",
        "db": "pmc",
        "id": pmid,
        "retmode": "xml"
    }
    r = requests.get(url, params=params)
    root = ET.fromstring(r.text)
    ids = root.findall(".//LinkSetDb/Link/Id")
    return f"PMC{ids[0].text}" if ids else None

In [5]:


def fetch_pmc_xml(pmcid):
    """Fetch PMC full text as XML via EFetch."""
    pmc_num = pmcid.replace("PMC", "")
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pmc",
        "id": pmc_num,
        "rettype": "full",
        "retmode": "xml",
        "tool": "amit_pmc_tool",
        "email": "amit.elia@ucsf.edu",
        # optionally "api_key": "...",
    }
    r = requests.get(url, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text


In [6]:
# -----------------------
# Example usage
# -----------------------

pmid = doi_to_pmid(doi)
print("PMID:", pmid)

if pmid:
    pmcid = pmid_to_pmcid(pmid)
    print("PMCID:", pmcid)

    if pmcid:
        html = fetch_pmc_xml(pmcid)
        if html:
            print("HTML length:", len(html))
        else:
            print("HTML returned blank.")
    else:
        print("No PMCID (paper not free on PMC).")
else:
    print("No PMID found (DOI not indexed by PubMed).")


PMID: 30952590
PMCID: PMC8375098


NameError: name 'HEADERS' is not defined

In [ ]:
print(html)

<?xml version="1.0"  ?><!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd"><pmc-articleset><article xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:ali="http://www.niso.org/schemas/ali/1.0/" xml:lang="EN" article-type="research-article" dtd-version="1.4"><front><journal-meta><journal-id journal-id-type="nlm-ta">ACS Omega</journal-id><journal-id journal-id-type="iso-abbrev">ACS Omega</journal-id><journal-id journal-id-type="pmc-domain-id">3686</journal-id><journal-id journal-id-type="pmc-domain">acsomega</journal-id><journal-id journal-id-type="publisher-id">ao</journal-id><journal-title-group><journal-title>ACS Omega</journal-title></journal-title-group><issn pub-type="epub">2470-1343</issn><publisher><publisher-name>American Chemical Society</publisher-name></publisher></journal-meta><article-meta><article-id pub-id-type="pmcid">PMC8375098</article-id><article-id pub-id-type="pmcid-ver">PMC8375098.

In [7]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import requests

HEADERS = {
    "User-Agent": "amit-pmc-scraper/0.1 (mailto:amit.elia@ucsf.edu)",
}

def fetch_pmc_fulltext(pmcid):
    """Download the full-text HTML for a PMCID."""
    url = f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}/"
    print(f"Accessing {url}")
    r = requests.get(url, headers=HEADERS, timeout=15)
    print("Status:", r.status_code, "Final URL:", r.url)
    if r.status_code == 200:
        return r.text
    print("Response snippet:", r.text[:400])
    return None

def extract_figure_image_urls_from_html(pmcid):
    """
    Parse the PMC article HTML and return a list of figure/scheme image URLs.

    Heuristics:
    - Look at <img> tags
    - Keep those whose alt text starts with "Figure" or "Scheme" (case insensitive)
    """
    html = fetch_pmc_fulltext(pmcid)
    if html is None:
        return []

    base_url = f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}/"
    soup = BeautifulSoup(html, "html.parser")

    urls = []
    for img in soup.find_all("img"):
        alt = (img.get("alt") or "").strip()
        src = img.get("src")
        if not src:
            continue

        # Focus on "real" figure/scheme images
        if alt.lower().startswith(("figure", "scheme")):
            full_url = urljoin(base_url, src)
            urls.append((alt, full_url))

    # Deduplicate by URL
    seen = set()
    unique_urls = []
    for alt, url in urls:
        if url not in seen:
            seen.add(url)
            unique_urls.append((alt, url))

    return unique_urls

In [8]:
pmcid = "PMC8375098"
fig_urls = extract_figure_image_urls_from_html(pmcid)
for alt, url in fig_urls:
    print(alt, "->", url)

Accessing https://pmc.ncbi.nlm.nih.gov/articles/PMC8375098/
Status: 200 Final URL: https://pmc.ncbi.nlm.nih.gov/articles/PMC8375098/
Figure 1 -> https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/8eee43b34906/ao1c03410_0002.jpg
Scheme 1 -> https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/c4d6539d7aaa/ao1c03410_0003.jpg
Scheme 2 -> https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/08ffe6a1d7fd/ao1c03410_0004.jpg


In [9]:
def download_images(fig_urls, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    paths = []

    for alt, url in fig_urls:
        r = requests.get(url, headers=HEADERS, timeout=30, stream=True)
        ctype = (r.headers.get("Content-Type") or "").lower()
        print("Downloading:", url, "| status:", r.status_code, "| content-type:", ctype)

        if r.status_code != 200 or not ctype.startswith("image/"):
            # If you find they're PDFs instead, you can allow application/pdf here.
            print("Skipping (not an image or bad status).")
            continue

        # Derive extension from content-type (e.g., image/jpeg -> .jpeg)
        ext = "." + ctype.split("/", 1)[1].split(";", 1)[0]
        # Make a reasonable filename stem from the alt text
        stem = alt.replace(" ", "_").replace(":", "").replace("/", "-")
        out_path = os.path.join(out_dir, stem + ext)

        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        print("Saved ->", out_path)
        paths.append(out_path)

    return paths

# -------------------
# Example: end-to-end
# -------------------
pmcid = "PMC8375098"
fig_urls = extract_figure_image_urls_from_html(pmcid)
for alt, url in fig_urls:
    print(alt, "->", url)

downloaded = download_images(fig_urls, out_dir=f"figures_{pmcid}")
print("Downloaded files:", downloaded)

Accessing https://pmc.ncbi.nlm.nih.gov/articles/PMC8375098/
Status: 200 Final URL: https://pmc.ncbi.nlm.nih.gov/articles/PMC8375098/
Figure 1 -> https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/8eee43b34906/ao1c03410_0002.jpg
Scheme 1 -> https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/c4d6539d7aaa/ao1c03410_0003.jpg
Scheme 2 -> https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/08ffe6a1d7fd/ao1c03410_0004.jpg
Downloading: https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/8eee43b34906/ao1c03410_0002.jpg | status: 200 | content-type: image/jpeg
Saved -> figures_PMC8375098\Figure_1.jpeg
Downloading: https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/c4d6539d7aaa/ao1c03410_0003.jpg | status: 200 | content-type: image/jpeg
Saved -> figures_PMC8375098\Scheme_1.jpeg
Downloading: https://cdn.ncbi.nlm.nih.gov/pmc/blobs/0d70/8375098/08ffe6a1d7fd/ao1c03410_0004.jpg | status: 200 | content-type: image/jpeg
Saved -> figures_PMC8375098\Scheme_2.jpeg
Downloaded files: ['figures_PMC8375098\